In [3]:
import napari
import imageio as io
from pathlib import Path
import numpy as np
import h5py
import pandas as pd
import histomicstk

In [2]:
features = ('label', 'area', 'area_convex', 'axis_major_length', 'axis_minor_length',  'centroid_local', 
    'eccentricity', 'equivalent_diameter_area', 'extent', 'perimeter','solidity')

In [3]:
from histomicstk.features import compute_gradient_features, compute_haralick_features, compute_fsd_features
from skimage.measure import regionprops_table
from skimage.segmentation import clear_border
from skimage.measure import label


def measure_morphology_2DT(embryo_id, labels, intensity_img):

    regionprops_list = []
    
    for timepoint, (label_timepoint, intensity_timepoint) in enumerate(zip(labels, intensity_img)):

        gradient_features = compute_gradient_features(label_timepoint, intensity_timepoint)
        haralick_features = compute_haralick_features(label_timepoint, intensity_timepoint)
        fsd_features = histomicstk.features.compute_fsd_features(label_timepoint)


        current_regionprops = {}
        current_regionprops['timepoint'] = timepoint
        regionprops_label = regionprops_table(label_image=label_timepoint, intensity_image=None, properties= features )

        test = label(label_timepoint)
        border_check = clear_border(test)
        if not np.array_equal(border_check, label_timepoint):
            current_regionprops['is_border'] = True

        current_regionprops.update(regionprops_label)
        current_regionprops.update(gradient_features)
        current_regionprops.update(haralick_features)
        current_regionprops.update(fsd_features)

        regionprops_list.append(pd.DataFrame(current_regionprops))  # Convert to DataFrame

    dataframe = pd.concat(regionprops_list)
    dataframe['embryo ID'] = embryo_id

    return dataframe


In [4]:

#loops over all the files with in a folder and creates one big dataset


fld = Path("Q:\carspi\ETiX-SBR-Cer1_exp1")

all_dfs=[]

for fn in fld.glob('*.h5'):
    with h5py.File(fn) as f:

        img= f['ch_00/1'][...]
        lbl = f['mask_cleanup_corrected/1'][...]
        

        embryo_id= fn.name.split('.')[0]
        embryo_id = embryo_id.split("_")[-1].strip('s')
        embryo_id = int(embryo_id)

        lbl_df=measure_morphology_2DT(embryo_id, lbl, img)
        all_dfs.append(lbl_df)

final_df = pd.concat(all_dfs, ignore_index=True)
# Assuming 'embryo ID' and 'timepoint' are the first two columns, and the rest are features
column_order = ['embryo ID', 'timepoint'] + [col for col in final_df.columns if col not in ['embryo ID', 'timepoint']]
final_df = final_df[column_order]

print(final_df)

      embryo ID  timepoint  label     area  area_convex  axis_major_length  \
0            10          0      1  45741.0      47388.0         250.910276   
1            10          1      1  54533.0      55264.0         271.369793   
2            10          2      1  60469.0      61821.0         286.696895   
3            10          3      1  64478.0      69363.0         294.895488   
4            10          4      1  65036.0      66082.0         307.951644   
...         ...        ...    ...      ...          ...                ...   
6803         99         18      1  58419.0      59900.0         313.491598   
6804         99         19      1  59701.0      61389.0         309.046673   
6805         99         20      1  61192.0      62296.0         310.750385   
6806         99         21      1  62887.0      64009.0         314.106298   
6807         99         22      1  65074.0      67181.0         312.606467   

      axis_minor_length  centroid_local-0  centroid_local-1  ec

In [5]:
final_df = final_df[final_df['label'] == 1]
final_df = final_df.drop(columns='label')

print(final_df)

      embryo ID  timepoint     area  area_convex  axis_major_length  \
0            10          0  45741.0      47388.0         250.910276   
1            10          1  54533.0      55264.0         271.369793   
2            10          2  60469.0      61821.0         286.696895   
3            10          3  64478.0      69363.0         294.895488   
4            10          4  65036.0      66082.0         307.951644   
...         ...        ...      ...          ...                ...   
6803         99         18  58419.0      59900.0         313.491598   
6804         99         19  59701.0      61389.0         309.046673   
6805         99         20  61192.0      62296.0         310.750385   
6806         99         21  62887.0      64009.0         314.106298   
6807         99         22  65074.0      67181.0         312.606467   

      axis_minor_length  centroid_local-0  centroid_local-1  eccentricity  \
0            233.422865        121.767036        127.919263      0.366

In [6]:
#df= final_df
# Identify 'embryo ID' values where 'is_border' is True
embryo_ids_to_remove = final_df.loc[final_df['is_border'] == True, 'embryo ID'].unique()

# Remove all rows with the identified 'embryo ID' values
filtered_df = final_df[~final_df['embryo ID'].isin(embryo_ids_to_remove)]

# Drop unnecessary columns
filtered_df = filtered_df.drop(['is_border'], axis=1)

# Print or inspect the filtered DataFrame
print(filtered_df)


      embryo ID  timepoint     area  area_convex  axis_major_length  \
69          106          0  32264.0      32993.0         235.723649   
70          106          1  33631.0      34302.0         242.115162   
71          106          2  34621.0      35361.0         245.460637   
72          106          3  36939.0      37628.0         257.880409   
73          106          4  38167.0      38692.0         266.193467   
...         ...        ...      ...          ...                ...   
6803         99         18  58419.0      59900.0         313.491598   
6804         99         19  59701.0      61389.0         309.046673   
6805         99         20  61192.0      62296.0         310.750385   
6806         99         21  62887.0      64009.0         314.106298   
6807         99         22  65074.0      67181.0         312.606467   

      axis_minor_length  centroid_local-0  centroid_local-1  eccentricity  \
69           175.364201        121.477250         93.158784      0.668

In [7]:
duplicate_rows = filtered_df.duplicated(subset=['embryo ID', 'timepoint'], keep=False)

duplicate_rows_data = filtered_df[duplicate_rows]
duplicate_rows_data

,embryo ID,timepoint,area,area_convex,axis_major_length,axis_minor_length,centroid_local-0,centroid_local-1,eccentricity,equivalent_diameter_area,...,Haralick.IMC1.Mean,Haralick.IMC1.Range,Haralick.IMC2.Mean,Haralick.IMC2.Range,Shape.FSD1,Shape.FSD2,Shape.FSD3,Shape.FSD4,Shape.FSD5,Shape.FSD6


In [9]:
fld_out = Path("Q:\carspi\ETiX_Output")

filtered_df.to_csv( fld_out /'240219_raw_features_full_df.csv', index=False)

In [4]:
df = pd.read_csv(r"Q:\carspi\ETiX_Output\240117_raw_features_full_dataset.csv")
df

,embryo ID,timepoint,area,area_convex,axis_major_length,axis_minor_length,centroid_local-0,centroid_local-1,eccentricity,equivalent_diameter_area,...,Haralick.IMC1.Mean,Haralick.IMC1.Range,Haralick.IMC2.Mean,Haralick.IMC2.Range,Shape.FSD1,Shape.FSD2,Shape.FSD3,Shape.FSD4,Shape.FSD5,Shape.FSD6
0,106,0,32264.0,32993.0,235.723649,175.364201,121.477250,93.158784,0.668247,202.681525,...,-0.033900,0.028199,0.518548,0.180877,0.365697,0.002497,0.004451,0.015286,0.064299,0.243633
1,106,1,33631.0,34302.0,242.115162,177.841359,124.198002,93.623710,0.678574,206.930711,...,-0.039054,0.025332,0.551364,0.149001,0.419379,0.001565,0.001841,0.012733,0.046679,0.224775
2,106,2,34621.0,35361.0,245.460637,180.685370,123.311314,86.448832,0.676865,209.954343,...,-0.042401,0.026222,0.571650,0.143042,0.339451,0.001278,0.003956,0.018690,0.073104,0.243937
3,106,3,36939.0,37628.0,257.880409,183.237527,132.601262,89.735916,0.703644,216.869075,...,-0.044629,0.027596,0.581277,0.144405,0.304059,0.005598,0.006374,0.042172,0.068877,0.234172
4,106,4,38167.0,38692.0,266.193467,183.078093,137.492729,90.809600,0.725935,220.444400,...,-0.050993,0.032595,0.612231,0.152366,0.044600,0.005647,0.010839,0.034391,0.133557,0.311358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2801,99,18,58419.0,59900.0,313.491598,238.726082,129.960903,147.465961,0.648157,272.729501,...,-0.091428,0.040896,0.746168,0.105661,0.445123,0.003127,0.005463,0.018671,0.093554,0.171988
2802,99,19,59701.0,61389.0,309.046673,247.711646,138.600023,143.694243,0.597948,275.705774,...,-0.103522,0.039851,0.777132,0.089391,0.480715,0.001046,0.009047,0.022269,0.064014,0.168311
2803,99,20,61192.0,62296.0,310.750385,252.075525,130.509756,153.088427,0.584792,279.127344,...,-0.103956,0.038293,0.777355,0.086472,0.425636,0.004613,0.007465,0.026800,0.090559,0.170039
2804,99,21,62887.0,64009.0,314.106298,255.950984,133.224037,154.567812,0.579665,282.966809,...,-0.110094,0.041684,0.791176,0.086347,0.247856,0.003768,0.009206,0.065973,0.097018,0.227144


In [5]:
filtered_df=df

In [6]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Pivot the DataFrame
multiindex_df = filtered_df.set_index(['embryo ID', 'timepoint'])

# Create a Scaler to normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))

# Group the DataFrame by "embryo ID"
grouped = multiindex_df.groupby(level=0, group_keys=False)

# Normalize each group and create a new DataFrame
normalized_df = grouped.apply(lambda group: pd.DataFrame(scaler.fit_transform(group), columns=group.columns, index=group.index))

# Reset the index to make "embryo ID" and "timepoint" regular columns
normalized_df.reset_index(inplace=True)

normalized_df

,embryo ID,timepoint,area,area_convex,axis_major_length,axis_minor_length,centroid_local-0,centroid_local-1,eccentricity,equivalent_diameter_area,...,Haralick.IMC1.Mean,Haralick.IMC1.Range,Haralick.IMC2.Mean,Haralick.IMC2.Range,Shape.FSD1,Shape.FSD2,Shape.FSD3,Shape.FSD4,Shape.FSD5,Shape.FSD6
0,106,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.114605,0.760254,0.000000,...,1.000000,0.211959,0.000000,1.000000,0.735855,0.276465,0.044038,0.015196,0.095851,0.716935
1,106,1,0.031236,0.028534,0.063624,0.021482,0.053010,0.122546,0.803173,0.039182,...,0.943347,0.000000,0.107740,0.724641,0.851554,0.145731,0.000000,0.000000,0.000000,0.638119
2,106,2,0.053858,0.051619,0.096926,0.046145,0.035734,0.000000,0.796070,0.067064,...,0.906546,0.065787,0.174341,0.673168,0.679285,0.105496,0.035687,0.035458,0.143748,0.718206
3,106,3,0.106825,0.101035,0.220557,0.068278,0.216736,0.056143,0.907362,0.130825,...,0.882052,0.167436,0.205949,0.684941,0.603006,0.711498,0.076481,0.175241,0.120750,0.677391
4,106,4,0.134886,0.124229,0.303308,0.066895,0.312040,0.074481,1.000000,0.163794,...,0.812100,0.537011,0.307575,0.753712,0.043794,0.718400,0.151822,0.128924,0.472592,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2801,99,18,0.806237,0.792647,0.992649,0.705172,0.734852,0.892561,0.740625,0.832137,...,0.304997,0.954173,0.801975,0.344218,0.922344,0.398290,0.271825,0.231381,0.652008,0.087308
2802,99,19,0.843563,0.835052,0.939496,0.800791,1.000000,0.835501,0.443354,0.865189,...,0.160838,0.893317,0.908772,0.159745,1.000000,0.044025,0.531355,0.289838,0.350887,0.071828
2803,99,20,0.886974,0.860882,0.959869,0.847229,0.751697,0.977619,0.365459,0.903187,...,0.155657,0.802633,0.909544,0.126657,0.879829,0.651441,0.416775,0.363472,0.621485,0.079100
2804,99,21,0.936324,0.909666,1.000000,0.888469,0.835003,1.000000,0.335105,0.945825,...,0.082498,1.000000,0.957212,0.125236,0.491946,0.507579,0.542876,1.000000,0.687321,0.319533


In [8]:
#normalize per timepoint
from sklearn.preprocessing import MinMaxScaler

df = filtered_df

# Pivot the DataFrame
multiindex_df = df.set_index(['embryo ID', 'timepoint'])

# Create a MinMaxScaler to normalize the data
scaler = MinMaxScaler(feature_range=(0,1))

# Group the DataFrame by "timepoint"
grouped = multiindex_df.groupby(level=1, group_keys=False)

# Normalize each group and create a new DataFrame
normalized_df = grouped.apply(lambda group: pd.DataFrame(scaler.fit_transform(group), columns=group.columns, index=group.index))

# Reset the index to make "embryo ID" and "timepoint" regular columns
normalized_df.reset_index(inplace=True)


In [9]:
fld_out = Path("Q:\carspi\ETiX_Output")

normalized_df.to_csv( fld_out /'240304_normalized_per_timepoint.csv', index=False)